# Добавление контекста в промпт большой языковой модели, для реализации чат-бота

Задачу использования в чат боте языковой моделью контекста (новых данных) без дообучения можно решить так:
1. Собрать датасет новых текстовых данных. Разбить его на части. Например на абзацы. Для каждого абзаца создать с помощью простой языковой модели (например BERT, LABSE) векторные представления.
2. Вычислить векторное представление текстового запроса пользователя той же моделью. Отобрать топ K похожих текстов сравнивая вектор запроса пользователя с векторами известных абзацев. Это будет контекстом запроса.
3. Составит промпт из К отобранных абзацев датасета и запроса пользователя. Отправить промпт в языковую модель обученную быть чатом, например LLama3.

Отбор похожих текстов нужен для того, чтобы уместить контекст в промпт языковой модели чат-бота (например LLama3). Т.к. обычно датасет дополнительной информации большой и весь не войдёт в промпт.

Это называется Retrieval-Augmented Generation (RAG) или Context Application?

Примерная схема решения:

<img src="https://habrastorage.org/r/w1560/getpro/habr/upload_files/a65/c81/973/a65c8197331c24fea7b7b4a5a0985795.png">

**Запуск сервера с моделью**

Коротко о OLLAMA: https://github.com/ivtipm/ML/blob/main/tools.md


1. Скачать OLLAMA
2. Скачать языковую модель (чат-бот), например gemma:7b
```bash
ollama pull gemma:7b
```
3. Запустить сервер OLLAMA
```bash
ollama serve
```

In [ ]:
# установка зависимостей

# deprecated: !pip install sentence-transformers langchain-community langchain  faiss-cpu faiss-gpu ollama
pip3 install -r requirements.txt

**LangChain**

Это пакет для взаимодействия с языковыми моделями разного рода (чат-боты, модели для генерации текста, модели выдающие эмбеддинги и т.п.).

Предоставляет высокоуровневые типы данных и функции для вышеописанных задач.




In [1]:
import pandas as pd
from langchain.document_loaders import DataFrameLoader                  # отдельный тип для хранения датафреймов
# будет разбивать тексты на части, чтобы делать их них эмбеддинги
from langchain.text_splitter import RecursiveCharacterTextSplitter      

In [2]:
# загрузка датасета, из которого будет использоваться информация для дополнения промпта

def load_dataset( filename:str ):
    """Загружает датасет из вопросов и ответов.
    Формат файла:
    Вопрос? <одна строка>
    Ответ,
    ответ может занимает несколько абзацев или строк
    @return: DataFrame(columns=['Q', 'A'])"""

    Q = []      # вопросы
    A = []      # ответы

    # загрузка вопросов и ответов из файла
    for line in open( filename ).readlines():
        line = line.strip()
        if line.endswith("?"):              # строка - это вопрос
            Q += [line]
            if len(A) > 0:
                A[-1] = A[-1].strip()
            A += [""]
        else:                               # строка - это ответ или его часть
            if len(A) > 0:
                A[-1] = A[-1] + " " + line
            else:
                A+= [line]

    data = pd.DataFrame( {"Q":Q, "A":A})
    return data

In [3]:
data = load_dataset("data.txt")

loader = DataFrameLoader(data, page_content_column='Q')         # зададим ключ для поиска по текстам, это колонка с вопросом
documents = loader.load()                                       # обёртка над датафреймом, отсюда будем брать контекст

data
# loader

,Q,A
0,Как мне принять участие/подать заявку на участ...,1. На сайте проекта https://hacks-ai.ru/ в тай...
1,"Я принимал участие в Конкурсе в 2021, 2022 и/и...",Для того чтобы принять участие в хакатоне ново...
2,"Как я узнаю, что моя команда допущена к участи...",Участники команд получат письмо с подтверждени...
3,"Я выбрал участие в одном окружном хакатоне, но...","Чтобы сменить хакатон, необходимо зайти в ЛК, ..."
4,"Я зарегистрировался, собрал команду. Что дальше?",Команда должна перейти в ЛК на сайте проекта в...
...,...,...
66,Вопросы по кейсу и трекерам можно задавать тол...,Любые вопросы по кейсу правильнее всего задава...
67,Когда и как команда должна предоставить разраб...,"Согласно расписанию хакатона, строго до дедлай..."
68,Каковы требования к разработанному решению (пр...,Созданный Участником Прототип не должен содерж...
69,Как будет происходить презентация проектов?,Структуру и регламент презентации решения кейс...


In [4]:
# класс-обёртка для создания эмбеддингов текстов
from langchain_community.embeddings import HuggingFaceEmbeddings

# сравнительно простая (и быстрая) модель, выдаёт эмбеддинги (векторы) для текстов
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings_maker = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    force_download=False,
)

/home/s/w/lab/py/my_venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/s/w/lab/py/my_venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:  50%|#####     | 220M/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# максимальная длина (в токенах) входной последовательности
# токен - слово, часть слова или буква
embeddings_maker.client.max_seq_length                  # 384

384

In [6]:
# размерность эмбеддинга 
embeddings_maker.client.get_sentence_embedding_dimension()

768

In [16]:
# нейросеть выдающая эмбеддинги текстов 
# embeddings_maker.client

In [7]:
# объект, который будет разбивать тексты из датасета на блоки, если вдруг они будут слишком большими для модели выдающий эмбеддинги
text_splitter = RecursiveCharacterTextSplitter(chunk_size = embeddings_maker.client.max_seq_length, chunk_overlap=0)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 50, chunk_overlap=0)        # для примера
# chunk_size - это размер блока в токенах, будет разбивать на части только ключ (здесь, это вопрос)

# получим блоки. Блок = (вопрос (ключ), ответ);
# Вопрос может быть не полным, если не поместится в chunk_size. Тогда создаётся новый блок, с остатком вопроса, но с таким же ответом.
texts = text_splitter.split_documents(documents)
len(texts)          # при максимальном размере вопроса в токенах 384, разбивать вопросы на части не пришлось.
# texts

71

In [8]:
# класс для хранения данных как в векторной БД?. Используется для быстрого поиска подходящего контекста по запросу
from langchain.vectorstores import FAISS

# создаем хранилище
db = FAISS.from_documents(texts, embeddings_maker)
db.as_retriever()           # ???ы

# пример использования:
db.similarity_search_with_score('Мне 10 лет. Я могу участвовать в хакатоне?', k = 5 )
# поданный запрос переводится в эмбеддинг, для него выдаётся топ K самых похожих частей датасета (вопрос, ответ, расстояние)

[(Document(page_content='Мне нет 14 лет, но я хочу принять участие в хакатоне, есть ли такая возможность?', metadata={'A': 'В хакатоне может принять участие физическое лицо, достигшее 14 лет. Если в команде окажется участник младше 14 лет, к сожалению, вся команда будет дисквалифицирована вне зависимости от занятого ею места в хакатоне.'}),
  0.23950584),
 (Document(page_content='Мне нет 14 лет, но я хочу принять участие в хакатоне, есть ли у меня такая возможность?', metadata={'A': 'В хакатоне может принять участие физическое лицо, достигшее 14 лет. Если в команде окажется участник младше 14 лет, к сожалению, вся команда будет дисквалифицирована вне зависимости от занятого ею места в хакатоне.  Как будут проходить окружные хакатоны'}),
  0.24520686),
 (Document(page_content='Могу ли я участвовать в нескольких хакатонах?', metadata={'A': 'Согласно Положению о проекте вы имеете право участвовать в нескольких хакатонах. Однако в случае занятия 1, 2 или 3 места на одном из окружных хакато

In [10]:
import ollama

# request = 'Мне 10 лет. Я могу участвовать в хакатоне?'
# request = 'Мне 100 лет. Я могу участвовать в хакатоне?'
# request = 'Сколько участников должно быть в команде?'
request = 'Есть ли чат в телеграмме? Если есть, то какой адрес?'

context = db.similarity_search_with_score(request, k = 5 )
context = " ".join([text[0].metadata['A'] for text in context])
context

response = ollama.chat(model='gemma:2b', messages=[
  {
    'role': 'user',
    'content': f'Дай развёрнутый и как можно более точный ответ. Для ответа используй дополнительную информацию.\nВопрос: {request}.\n Дополнительная информация: {context}',}],
    stream = True         # отвечать последовательно выдавая текст, а не отдельным готовым блоком текста
)
# print(response['message']['content'])

for chunk in response:
  print(chunk['message']['content'], end='', flush=True)

**Есть чат в телеграмме.**

Канал телеграмме имеет название **"Россия — страна возможностей"**.

**Адрес:** chat.russia.org

**Способ доступа:**

1. Зайдите в приложение Telegram.
2. Введите название канала **"Россия — страна возможностей"**.
3. Выберите свою учетную запись.
4. Настройте чат.

**Важно:**

* Вам необходимо иметь активную учетную запись на платформе Telegram.
* Для того, чтобы создать чат, вам необходимо быть по возрасту 18 лет.